In [2]:
import numpy as np
import pandas as pd

from glob import glob
from skimage import measure

from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb


import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

from features import get_region_from_map, get_features_from_region

In [2]:
data_path = "/home/samsmu/Data/Lung/CT"
working_path = data_path + "/DSB3_work"

### 0. Helper functions

In [3]:
def getRegionMetricRow(seg):
    # fname, numpy array of dimension [#slices, 1, 512, 512] containing the images
    nslices = seg.shape[0]
    
    #metrics
    totalArea = 0.
    avgArea = 0.
    maxArea = 0.
    avgEcc = 0.
    avgEquivlentDiameter = 0.
    stdEquivlentDiameter = 0.
    weightedX = 0.
    weightedY = 0.
    numNodes = 0.
    numNodesperSlice = 0.
    # crude hueristic to filter some bad segmentaitons
    # do not allow any nodes to be larger than 10% of the pixels to eliminate background regions
    maxAllowedArea = 0.10 * 512 * 512 
    
    areas = []
    eqDiameters = []
    for slicen in range(nslices):
        regions = getRegionFromMap(seg[slicen])
        for region in regions:
            if region.area > maxAllowedArea:
                continue
            totalArea += region.area
            areas.append(region.area)
            avgEcc += region.eccentricity
            avgEquivlentDiameter += region.equivalent_diameter
            eqDiameters.append(region.equivalent_diameter)
            weightedX += region.centroid[0]*region.area
            weightedY += region.centroid[1]*region.area
            numNodes += 1
            
    weightedX = weightedX / totalArea 
    weightedY = weightedY / totalArea
    avgArea = totalArea / numNodes
    avgEcc = avgEcc / numNodes
    avgEquivlentDiameter = avgEquivlentDiameter / numNodes
    stdEquivlentDiameter = np.std(eqDiameters)
    
    maxArea = max(areas)
    
    numNodesperSlice = numNodes*1. / nslices
    
    return np.array([avgArea, maxArea, avgEcc, avgEquivlentDiameter, stdEquivlentDiameter, weightedX, weightedY, numNodes, numNodesperSlice])


### 1. Form Dataset full series

In [ ]:
#%%time
file_list = glob(working_path + "/nodules*")
labels_file = data_path + '/DSB3/stage1_labels.csv'
df_labels = pd.read_csv(labels_file)
numfeatures = 9
feature_array = np.zeros((len(file_list),numfeatures))
labels = np.zeros((len(file_list)))
for i, fname in enumerate(file_list):
    series_id = fname.split('_')[-1].split('.')[0]
    print(series_id)
    series = np.load(fname) 
    feature_array[i] = getRegionMetricRow(series)
    labels[i] = df_labels[df_labels['id'] == series_id].iloc[0]['cancer']
np.save("data_series_X.npy", feature_array)
np.save("data_series_Y.npy", labels)



### 2. Form Dataset nodules

In [ ]:
#1 hour
%%time
file_list = glob(working_path + "/nodules*")
labels_file = data_path + '/DSB3/stage1_labels.csv'
df_labels = pd.read_csv(labels_file)
maxAllowedArea = 0.10 * 512 * 512 

feature_array = []
labels = [] 

for i, fname in enumerate(file_list):
    series_id = fname.split('_')[-1].split('.')[0]
    print(series_id)
    series = np.load(fname) 
    series_label = df_labels[df_labels['id'] == series_id].iloc[0]['cancer']    
    for seg in series:
        regions = getRegionFromMap(seg)
        for region in regions:
            if region.area > maxAllowedArea:
                continue
            
            nodule_features = get_features_from_region(region)
            feature_array.append(nodule_features)
            labels.append(series_label)

np.save("data_nodules_X.npy", np.array(feature_array))
np.save("data_nodules_Y.npy", np.array(labels))

### End

In [58]:
 np.array(sereslabels)

array([<function label at 0x7f28901101f0>,
       <function label at 0x7f28901101f0>,
       <function label at 0x7f28901101f0>, ...,
       <function label at 0x7f28901101f0>,
       <function label at 0x7f28901101f0>,
       <function label at 0x7f28901101f0>], dtype=object)